In [1]:
import numpy
import pandas
import custom
import pickle
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from IPython.display import display


In [2]:
##데이터 불러오기
df = pandas.read_csv("data/simple_movie_data.csv", encoding="UTF8", index_col=0)
display(df)

with open("data/vector.pkl", mode="rb") as f:
    vec = pickle.load(f)

print(len(vec))

,quote,scoreSentiment,word_len
24,if you haven t seen it do,1,7
39,it s benchmark cinema a highpoint of movie his...,1,9
47,this movie is all about brando,1,6
63,this is sean connery at his best,1,7
72,there are so many questions that the script ne...,0,10
...,...,...,...
967411,technically a little bit crude but it delivers,1,8
967412,a sturdy N entertaining late night watch N,1,8
967492,what on earth happened here N,0,6
967540,is it tuesday yet,0,4


123977


In [3]:
##데이터 전처리

sentences = df["quote"].values.tolist()
t = df["scoreSentiment"].values.tolist()
word_len = df["word_len"].max()

x = []
for s in sentences :
    temp = []
    words = custom.text_preprocess(s).split()
    for i in range(word_len) :
        if i >= len(words) :
            temp.append(vec["<pad>"])
            continue
        if words[i] not in vec :
            temp.append(vec["<unk>"])
            continue
        temp.append(vec[words[i]])
    x.append(temp)
print(len(x))
    

101086


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

##DataLoader 생성
tensor_x = torch.tensor(x, dtype = torch.float, device = device)
tensor_t = torch.tensor(t, dtype = torch.long, device = device)
#zip 함수는 parameter로 쓰는 자료들도 자료형 바꾸어버림
val_x = tensor_x
val_t = tensor_t
dataloader = DataLoader(list(zip(tensor_x, tensor_t)), batch_size= 100, shuffle= True)



In [14]:
##AI 만들기
#함수 선언
class NN(nn.Module) :
    def __init__(self) :
        super().__init__()
        self.rnn = nn.RNN(300,10,batch_first = True,device = device)
        self.f = nn.Sequential(
            nn.Linear(10,2),
            nn.ReLU()
        )
    def forward(self, x) :
        x, h = self.rnn(x)
        x = x[:,-1,:]
        x = self.f(x)
        return x
F = NN()
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(F.parameters(), lr = 0.15) #tanh 쓸 시, lr = 0.15
# optimizer = torch.optim.SGD(F.parameters(), lr = 0.05) #relu 쓸 시, lr = 0.05
epoch = 50

for e in range(epoch) :
    loss_sum = 0
    for x, t in dataloader :
    # x, t 입력
    # y = F(x)
        y = F(x)
    # 손실함수
        loss = loss_function(y,t)
        loss_sum += loss
    # 최적화함수
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"epoch {e+1} || loss {loss_sum / len(dataloader)}")

epoch 1 || loss 0.6472021341323853
epoch 2 || loss 0.549453616142273
epoch 3 || loss 0.5075717568397522
epoch 4 || loss 0.48564475774765015
epoch 5 || loss 0.4709601104259491
epoch 6 || loss 0.458795428276062
epoch 7 || loss 0.4524785280227661
epoch 8 || loss 0.4456425607204437
epoch 9 || loss 0.4424932897090912
epoch 10 || loss 0.438081830739975
epoch 11 || loss 0.432682603597641
epoch 12 || loss 0.4315200746059418
epoch 13 || loss 0.4279804825782776
epoch 14 || loss 0.42577803134918213
epoch 15 || loss 0.423071026802063
epoch 16 || loss 0.42062297463417053
epoch 17 || loss 0.41972869634628296
epoch 18 || loss 0.4169659912586212
epoch 19 || loss 0.4154372215270996
epoch 20 || loss 0.414175420999527
epoch 21 || loss 0.4127240777015686
epoch 22 || loss 0.4110766053199768
epoch 23 || loss 0.4095962643623352
epoch 24 || loss 0.4081116020679474
epoch 25 || loss 0.4062221944332123
epoch 26 || loss 0.40604373812675476
epoch 27 || loss 0.4037821590900421
epoch 28 || loss 0.4042403995990753
ep

In [15]:
cnt = 0
for i in range(val_x.shape[0]) :
    y =F(val_x[i:i+1])
    if torch.argmax(y) == val_t[i] :
        cnt += 1

print("possiblity : %f" %(cnt / val_x.shape[0]))

possiblity : 0.822419


In [16]:
torch.save(F, "simple_movie_tanh.pt")